In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.ensemble import StackingRegressor

In [3]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline
from catboost import CatBoostRegressor


In [19]:
df=pd.read_csv('properties.csv')
df.head()

,id,price,property_type,subproperty_type,region,province,locality,zip_code,latitude,longitude,...,fl_garden,garden_sqm,fl_swimming_pool,fl_floodzone,state_building,primary_energy_consumption_sqm,epc,heating_type,fl_double_glazing,cadastral_income
0,34221000,225000.0,APARTMENT,APARTMENT,Flanders,Antwerp,Antwerp,2050,51.217172,4.379982,...,0,0.0,0,0,MISSING,231.0,C,GAS,1,922.0
1,2104000,449000.0,HOUSE,HOUSE,Flanders,East Flanders,Gent,9185,51.174944,3.845248,...,0,0.0,0,0,MISSING,221.0,C,MISSING,1,406.0
2,34036000,335000.0,APARTMENT,APARTMENT,Brussels-Capital,Brussels,Brussels,1070,50.842043,4.334543,...,0,0.0,0,1,AS_NEW,NaN,MISSING,GAS,0,NaN
3,58496000,501000.0,HOUSE,HOUSE,Flanders,Antwerp,Turnhout,2275,51.238312,4.817192,...,0,0.0,0,1,MISSING,99.0,A,MISSING,0,NaN
4,48727000,982700.0,APARTMENT,DUPLEX,Wallonia,Walloon Brabant,Nivelles,1410,NaN,NaN,...,1,142.0,0,0,AS_NEW,19.0,A+,GAS,0,NaN


In [5]:
df_mod=df.drop(columns=['fl_floodzone','fl_swimming_pool','fl_garden','fl_terrace','fl_open_fire','fl_furnished','fl_double_glazing','locality'])

In [6]:
# Filtering by house
df_houses = df_mod[df_mod['subproperty_type'] == 'HOUSE']
df_houses.head()

,id,price,property_type,subproperty_type,region,province,zip_code,latitude,longitude,construction_year,...,nbr_frontages,nbr_bedrooms,equipped_kitchen,terrace_sqm,garden_sqm,state_building,primary_energy_consumption_sqm,epc,heating_type,cadastral_income
1,2104000,449000.0,HOUSE,HOUSE,Flanders,East Flanders,9185,51.174944,3.845248,NaN,...,NaN,2.0,MISSING,0.0,0.0,MISSING,221.0,C,MISSING,406.0
3,58496000,501000.0,HOUSE,HOUSE,Flanders,Antwerp,2275,51.238312,4.817192,2024.0,...,NaN,3.0,MISSING,0.0,0.0,MISSING,99.0,A,MISSING,NaN
5,22183000,548514.0,HOUSE,HOUSE,Flanders,Flemish Brabant,1700,NaN,NaN,NaN,...,4.0,3.0,MISSING,0.0,0.0,AS_NEW,NaN,MISSING,MISSING,NaN
7,48707000,424000.0,HOUSE,HOUSE,Flanders,East Flanders,9800,50.983015,3.570742,2023.0,...,NaN,4.0,HYPER_EQUIPPED,0.0,0.0,GOOD,NaN,MISSING,MISSING,NaN
10,28322000,385000.0,HOUSE,HOUSE,Flanders,East Flanders,9190,51.206244,4.075700,NaN,...,3.0,3.0,MISSING,0.0,0.0,MISSING,NaN,MISSING,MISSING,NaN


In [7]:
import numpy as np

# Define lists to store numerical and categorical column names
numerical_cols = []
categorical_cols = []

# Loop through DataFrame columns
for column in df_houses.columns:
    # Check if the column is numerical
    if df_houses[column].dtype in [np.int64, np.float64]:
        numerical_cols.append(column)
    else:
        categorical_cols.append(column)

# Print the lists of numerical and categorical columns
print("Numerical columns:", numerical_cols)
print("Categorical columns:", categorical_cols)

# Print the index of numerical and categorical columns
print("Index of numerical columns:", [df_houses.columns.get_loc(col) for col in numerical_cols])
print("Index of categorical columns:", [df_houses.columns.get_loc(col) for col in categorical_cols])



Numerical columns: ['id', 'price', 'zip_code', 'latitude', 'longitude', 'construction_year', 'total_area_sqm', 'surface_land_sqm', 'nbr_frontages', 'nbr_bedrooms', 'terrace_sqm', 'garden_sqm', 'primary_energy_consumption_sqm', 'cadastral_income']
Categorical columns: ['property_type', 'subproperty_type', 'region', 'province', 'equipped_kitchen', 'state_building', 'epc', 'heating_type']
Index of numerical columns: [0, 1, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 18, 21]
Index of categorical columns: [2, 3, 4, 5, 14, 17, 19, 20]


In [8]:
from sklearn.model_selection import KFold
X = df_houses.drop(columns=['price'])
y = df_houses[['price']]  # Target variable
# Define the number of splits for Stratified K-Fold Cross-Validation
n_splits = 5

# Initialize the KFold object
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=0)

# Iterate over the splits and obtain the training and testing indices
for train_index, test_index in kfold.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Printing the shapes of the training and testing sets for each fold
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (23784, 21)
Shape of X_test: (5946, 21)
Shape of y_train: (23784, 1)
Shape of y_test: (5946, 1)


In [9]:
columns_to_drop = ['id', 'property_type', 'subproperty_type']
X_train=X_train.drop(columns=columns_to_drop)
X_test=X_test.drop(columns=columns_to_drop)



In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Define the numerical and categorical columns
numerical_cols = ['zip_code', 'latitude', 'longitude', 'construction_year', 'total_area_sqm', 'surface_land_sqm', 'nbr_frontages', 'nbr_bedrooms', 'terrace_sqm', 'garden_sqm', 'primary_energy_consumption_sqm', 'cadastral_income']
categorical_cols = [ 'region','province', 'equipped_kitchen', 'state_building', 'epc', 'heating_type']

# Create a ColumnTransformer with KNNImputer and StandardScaler for numerical columns, and OneHotEncoder for categorical columns
numerical_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ],
    remainder='passthrough'  # passthrough any columns not being transformed
)



# Define CatBoostRegressor
cb_regressor = CatBoostRegressor()

# Define the pipeline
pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', cb_regressor)
])
# Fit the pipeline with X_train and y_train
pipe.fit(X_train, y_train)


Learning rate set to 0.067551
0:	learn: 316973.9168576	total: 206ms	remaining: 3m 26s
1:	learn: 309886.5461011	total: 218ms	remaining: 1m 48s
2:	learn: 302590.7185646	total: 231ms	remaining: 1m 16s
3:	learn: 295850.5662038	total: 242ms	remaining: 1m
4:	learn: 289857.1341117	total: 249ms	remaining: 49.6s
5:	learn: 284703.3753158	total: 258ms	remaining: 42.8s
6:	learn: 278833.5702888	total: 266ms	remaining: 37.7s
7:	learn: 273448.6219845	total: 276ms	remaining: 34.2s
8:	learn: 268691.4676471	total: 284ms	remaining: 31.3s
9:	learn: 264413.5348935	total: 291ms	remaining: 28.8s
10:	learn: 260309.8389157	total: 298ms	remaining: 26.8s
11:	learn: 256611.8003744	total: 306ms	remaining: 25.2s
12:	learn: 253316.0002056	total: 312ms	remaining: 23.7s
13:	learn: 249926.5311561	total: 319ms	remaining: 22.5s
14:	learn: 246651.5799531	total: 327ms	remaining: 21.5s
15:	learn: 244105.2684805	total: 335ms	remaining: 20.6s
16:	learn: 242019.4078106	total: 344ms	remaining: 19.9s
17:	learn: 241639.0543802	to

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['zip_code', 'latitude',
                                                   'longitude',
                                                   'construction_year',
                                                   'total_area_sqm',
                                                   'surface_land_sqm',
                                                   'nbr_frontages',
                                                   'nbr_bedrooms',
                                                   'terrace_sqm', 'garden_sqm',
                                                   'primary_energy_consumption_sqm',
                                                   'cadastral_income']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['region', 'province',
                                                   'equipped_kitchen',
                                                   'state_building', 'epc',
                                                   'heating_type'])])),
                ('regressor',
                 <catboost.core.CatBoostRegressor object at 0x0000015AF7249220>)])

In [11]:
train_score = pipe.score(X_train, y_train)
print("Model score on training set:", train_score)
test_score = pipe.score(X_test, y_test)
print("Model score on training set:", test_score)

Model score on training set: 0.9365708442108873
Model score on training set: 0.7533732920852477


In [12]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Make predictions on the test set
y_pred = pipe.predict(X_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)


# Print evaluation metrics
# Print evaluation metrics
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)





Mean Absolute Error: 68331.74265001719
Mean Squared Error: 27027167311.646053


In [13]:
import pickle

# Assuming you have already trained your model and named it 'pipe'
# Save the model to a file
with open('trained_model.pkl', 'wb') as file:
    pickle.dump(pipe, file)


In [14]:
import os

# Path to the saved model file
model_file_path = 'trained_model.pkl'

# Check if the model file exists
if os.path.exists(model_file_path):
    # Get the size of the file
    file_size = os.path.getsize(model_file_path)
     # Convert bytes to megabytes
    file_size_mb = file_size / (1024 * 1024)  # 1 MB = 1024 * 1024 bytes
    
    print("Model saved successfully.")
    print(f"Size of the model file: {file_size_mb:.2f} MB")
else:
    print("Model was not saved successfully.")


Model saved successfully.
Size of the model file: 3.51 MB


In [15]:
import pickle

# Load the saved model from the pickle file
with open('trained_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

# Make predictions using the loaded model
# Assuming X_test is your test dataset
y_pred = loaded_model.predict(X_test)



# Define the new data sample as a dictionary
new_data = { 
    
    'region': ['Flanders'],
    'province': ['Antwerp'],
    'locality': ['Antwerp'],
    'zip_code': [2050],
    'latitude': [51.217172],
    'longitude': [4.379982],
    'construction_year': [2020],
    'total_area_sqm': [100],
    'surface_land_sqm': [200],
    'nbr_frontages': [2],
    'nbr_bedrooms': [3],
    'equipped_kitchen': ['HYPER_EQUIPPED'],
    'terrace_sqm': [20],
    'garden_sqm': [50],
    'state_building': ['New'],
    'primary_energy_consumption_sqm': [150],
    'epc': ['C'],
    'heating_type': ['GAS'],
    'cadastral_income': [1000]
}


# Convert the dictionary to a DataFrame
X_new = pd.DataFrame(new_data)


# Predict the price using the loaded model
predicted_price = loaded_model.predict(X_new)

# Print the predicted price
print("Predicted price:", predicted_price)



Predicted price: [419794.36689731]


c:\Users\becod\GNT-Arai-6\immo-liza-deployment\myenv\Lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [16]:
'''import pickle
import pandas as pd

# Load the saved model from the pickle file
with open('trained_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

# Define the new data sample as a dictionary
new_data = {}

# Ask for user input for each feature

new_data['region'] = input("Enter region: ")
new_data['province'] = input("Enter province: ")
new_data['locality'] = input("Enter locality: ")
new_data['zip_code'] = int(input("Enter ZIP code: "))
new_data['latitude'] = float(input("Enter latitude: "))
new_data['longitude'] = float(input("Enter longitude: "))
new_data['construction_year'] = int(input("Enter construction year: "))
new_data['total_area_sqm'] = float(input("Enter total area in square meters: "))
new_data['surface_land_sqm'] = float(input("Enter surface land area in square meters: "))
new_data['nbr_frontages'] = int(input("Enter number of frontages: "))
new_data['nbr_bedrooms'] = int(input("Enter number of bedrooms: "))
new_data['equipped_kitchen'] = input("Enter equipped kitchen type (e.g., HYPER_EQUIPPED, MISSING): ")
new_data['terrace_sqm'] = float(input("Enter terrace area in square meters: "))
new_data['garden_sqm'] = float(input("Enter garden area in square meters: "))
new_data['state_building'] = input("Enter state of building (e.g., New, AS_NEW, MISSING): ")
new_data['primary_energy_consumption_sqm'] = float(input("Enter primary energy consumption in square meters: "))
new_data['epc'] = input("Enter EPC rating (e.g., A, B, C): ")
new_data['heating_type'] = input("Enter heating type (e.g., GAS, ELECTRICITY): ")
new_data['cadastral_income'] = float(input("Enter cadastral income: "))

# Convert the dictionary to a DataFrame
X_new = pd.DataFrame(new_data, index=[0])

# Predict the price using the loaded model
predicted_price = loaded_model.predict(X_new)

# Print the predicted price
print("Predicted price:", predicted_price)'''


'import pickle\nimport pandas as pd\n\n# Load the saved model from the pickle file\nwith open(\'cat_model.pkl\', \'rb\') as file:\n    loaded_model = pickle.load(file)\n\n# Define the new data sample as a dictionary\nnew_data = {}\n\n# Ask for user input for each feature\n\nnew_data[\'region\'] = input("Enter region: ")\nnew_data[\'province\'] = input("Enter province: ")\nnew_data[\'locality\'] = input("Enter locality: ")\nnew_data[\'zip_code\'] = int(input("Enter ZIP code: "))\nnew_data[\'latitude\'] = float(input("Enter latitude: "))\nnew_data[\'longitude\'] = float(input("Enter longitude: "))\nnew_data[\'construction_year\'] = int(input("Enter construction year: "))\nnew_data[\'total_area_sqm\'] = float(input("Enter total area in square meters: "))\nnew_data[\'surface_land_sqm\'] = float(input("Enter surface land area in square meters: "))\nnew_data[\'nbr_frontages\'] = int(input("Enter number of frontages: "))\nnew_data[\'nbr_bedrooms\'] = int(input("Enter number of bedrooms: "))\n

In [17]:
'''from fastapi import FastAPI
import pickle

# 1. Create a FastAPI application
app = FastAPI()

# 2. Load the trained model
with open("trained_model.pkl", "rb") as f:
    model = pickle.load(f)

# 3. Define API endpoints
@app.post("/predict")
async def predict(data: dict):
    # 4. Implement model prediction
    features = data["features"]  # Assuming features are passed in the request
    prediction = model.predict(features)
    return {"prediction": prediction}
'''



'from fastapi import FastAPI\nimport pickle\n\n# 1. Create a FastAPI application\napp = FastAPI()\n\n# 2. Load the trained model\nwith open("trained_model.pkl", "rb") as f:\n    model = pickle.load(f)\n\n# 3. Define API endpoints\n@app.post("/predict")\nasync def predict(data: dict):\n    # 4. Implement model prediction\n    features = data["features"]  # Assuming features are passed in the request\n    prediction = model.predict(features)\n    return {"prediction": prediction}\n'

In [18]:
'''import requests

# Define the URL of your FastAPI application's prediction endpoint
url = 'http://127.0.0.1:8000/'  

# Define the data to be sent in the POST request
data = {
    'features': {
        'subproperty_type': ['APARTMENT'],
    'region': ['Flanders'],
    'province': ['Antwerp'],
    'locality': ['Antwerp'],
    'zip_code': [2050],
    'latitude': [51.217172],
    'longitude': [4.379982],
    'construction_year': [2020],
    'total_area_sqm': [100],
    'surface_land_sqm': [200],
    'nbr_frontages': [2],
    'nbr_bedrooms': [3],
    'equipped_kitchen': ['HYPER_EQUIPPED'],
    'terrace_sqm': [20],
    'garden_sqm': [50],
    'state_building': ['New'],
    'primary_energy_consumption_sqm': [150],
    'epc': ['C'],
    'heating_type': ['GAS'],
    'cadastral_income': [1000]
       
    }
}

# Send the POST request
response = requests.post(url, json=data)

# Check the response
if response.status_code == 200:
    # Successful request
    result = response.json()
    print("Prediction:", result['prediction'])
else:
    # Failed request
    print("Failed to make prediction. Status code:", response.status_code)'''


'import requests\n\n# Define the URL of your FastAPI application\'s prediction endpoint\nurl = \'http://127.0.0.1:8000/\'  \n\n# Define the data to be sent in the POST request\ndata = {\n    \'features\': {\n        \'subproperty_type\': [\'APARTMENT\'],\n    \'region\': [\'Flanders\'],\n    \'province\': [\'Antwerp\'],\n    \'locality\': [\'Antwerp\'],\n    \'zip_code\': [2050],\n    \'latitude\': [51.217172],\n    \'longitude\': [4.379982],\n    \'construction_year\': [2020],\n    \'total_area_sqm\': [100],\n    \'surface_land_sqm\': [200],\n    \'nbr_frontages\': [2],\n    \'nbr_bedrooms\': [3],\n    \'equipped_kitchen\': [\'HYPER_EQUIPPED\'],\n    \'terrace_sqm\': [20],\n    \'garden_sqm\': [50],\n    \'state_building\': [\'New\'],\n    \'primary_energy_consumption_sqm\': [150],\n    \'epc\': [\'C\'],\n    \'heating_type\': [\'GAS\'],\n    \'cadastral_income\': [1000]\n       \n    }\n}\n\n# Send the POST request\nresponse = requests.post(url, json=data)\n\n# Check the response\nif